# Partisan web scraping

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import re
import json
import pandas as pd
import time
from datetime import date
import os
import requests
import tqdm
import ipywidgets
import time
from multiprocessing import Pool, set_start_method
from download_partisans_html import download_partisans_html # Этот файл лежит в репозитории отдельно

In [4]:
# Командадля распараллеливания
set_start_method('spawn')

In [ ]:
# Краулер, сохраняющий html файлы в /links/number.htm
if __name__ == '__main__':
    p = Pool(30)
    numbers_considered = range(23000, 170000) # Задаю диапазон
    number_processed = []
    for number in tqdm.tqdm(p.imap(download_partisans_html,
                                   numbers_considered),
                            total=len(numbers_considered)):
        number_processed.append(number)

In [ ]:
# Устанавливаю сегодняшнюю дату
today = str(date.today())
today

In [ ]:
# Основной парсер для партизан
partisans = []
links = os.listdir("/Users/annalevina/Desktop/links")
z = 1
for link in links:
    if link == '.DS_Store':
        continue
    with open(os.path.join('/Users/annalevina/Desktop/links', link)) as f:
        soup = BeautifulSoup(f.read()) # Считываю файлы из папки
    credentials = {}
    name = soup.find("h1", attrs={'class': "info-book__title"}).text.strip()
    name = name.split(sep=" ", maxsplit=3)
    bio = soup.find("ul", attrs={'class': "info-book__descr"}).text
    credentials["original_link"] = "https://partizany.by/partisans/" + link + "/"
    credentials["id"] = "partisans-" + str(z)
    z = z + 1 # Задаю ID для каждого партизана
    credentials["retrieved_at"] = today
    try:
        credentials["surname"] = name[0]
    except IndexError:
        credentials["surname"] = ""
    try:
        credentials["firstname"] = name[1]
    except IndexError:
        credentials["firstname"] = ""
    try:
        credentials["patronym"] = name[2]
    except IndexError:
        credentials["patronym"] = ""
    try:
        credentials["fullname"] = name[0] + " " + name[1] + " " + name[2]
    except IndexError:
        credentials["fullname"] = ""
    try:
        credentials["birthdate"] = re.search(r'\d{4}', bio).group().strip()
    except AttributeError:
        credentials["birthdate"] = ""
    try:
        credentials["birthplace"] = re.search(r'(?<=Место\sрождения: \s).*',
                                              bio).group().strip()
    except AttributeError:
        credentials["birthplace"] = ""
    try:
        credentials["nationality"] = re.search(r'(?<=Национальность:\s).*',
                                               bio).group().strip()
    except AttributeError:
        credentials["nationality"] = ""
    try:
        credentials["award"] = []
        award = soup.find("div", attrs={'class': "info-book__rewards-list"})
        # Следующим шагом использую цикл, потому что в одной карточке партизана может быть несколько наград
        for rewards_item in award.find_all("div", attrs={'class': "info-book__rewards-item"}):
            credentials["award"].append(rewards_item.find("img")['alt'])
    except AttributeError:
        credentials["award"] = ""
    try:
        credentials["award_nomination"] = []
        presented = soup.find("div", attrs={'class': "info-book__rewards-inline"})
        # Цикл, потому что нагарад может быть несколько
        for item in presented.find_all("span", attrs={'class': "info-book__rewards-inline-text"}):
            credentials["award_nomination"].append(item.text)
    except AttributeError:
        credentials["award_nomination"] = ""
    try:
        credentials["partisan_time_period"] = []
        credentials["partisan_brigade"] = []
        credentials["partisan_detachment"] = []
        credentials["partisan_position"] = []
        # Каждый партизан может состоять в нескольких формированиях и занимать в них разные должности
        formation__list = soup.find("div", attrs={'class': "formation__list"})
        for formation_item in formation__list.find_all("div", attrs={'class': "formation__item"}):
            credentials["partisan_time_period"].append(
                formation_item.find("div", attrs={'class': "formation__item-date"}).text)
        for item_descr in formation_item.find_all("ul", attrs={'class': "formation__item-descr"}):
            help = item_descr.find_all("li")
            credentials["partisan_brigade"].append(help[0].find('a').text)
            credentials["partisan_detachment"].append(help[1].find('a').text)
            credentials["partisan_position"].append(
                re.sub("^\nДолжность: ", "", help[2].text).strip())
    except:
        credentials["partisan_time_period"] = ""
        credentials["partisan_brigade"] = ""
        credentials["partisan_detachment"] = ""
        credentials["partisan_position"] = ""
    partisans.append(credentials)

In [35]:
# Запись в файл
with open('partisans.json', 'w', encoding='utf-8') as f:
    json.dump(partisans, f, ensure_ascii=False, indent=4)